# ETL - YELP!

### Librerías

In [3]:
import pandas as pd
# Se configura pandas para que se muestre la totalidad de las columnas del dataframe.
pd.options.display.max_columns = None
import ast
import json
import pyarrow as pa
import pyarrow.parquet as pq

### Rutas a los archivos.

In [4]:
path_buisness = 'C:\\Users\\fedez\\OneDrive\\Escritorio\\ProyectoG4-Google_Yelp\\Data\\Raw\\business.pkl'
path_checkin = 'C:\\Users\\fedez\\OneDrive\\Escritorio\\ProyectoG4-Google_Yelp\\Data\\Raw\\checkin.json'
path_review = 'C:\\Users\\fedez\\OneDrive\\Escritorio\\ProyectoG4-Google_Yelp\\Data\\Raw\\review.json'
path_review_parquet = "C:\\Users\\fedez\\OneDrive\\Escritorio\\ProyectoG4-Google_Yelp\\Data\\Raw\\review.parquet"
path_tip = 'C:\\Users\\fedez\\OneDrive\\Escritorio\\ProyectoG4-Google_Yelp\\Data\\Raw\\tip.json'
path_user = 'C:\\Users\\fedez\\OneDrive\\Escritorio\\ProyectoG4-Google_Yelp\\Data\\Raw\\user.parquet'

## Archivo "business.pkl"

In [5]:
df_business = pd.read_pickle(path_buisness)

In [6]:
df_business.head(4)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,NaN,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,NaN,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,NaN,85711,32.223236,-110.880452,3.5,22,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,CA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
df_business.info()

<class 'pandas.core.frame.DataFrame'>
Index: 150346 entries, 0 to 150345
Data columns (total 28 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   business_id   150346 non-null  object
 1   name          150346 non-null  object
 2   address       150346 non-null  object
 3   city          150346 non-null  object
 4   state         150343 non-null  object
 5   postal_code   150346 non-null  object
 6   latitude      150346 non-null  object
 7   longitude     150346 non-null  object
 8   stars         150346 non-null  object
 9   review_count  150346 non-null  object
 10  is_open       150346 non-null  object
 11  attributes    136602 non-null  object
 12  categories    150243 non-null  object
 13  hours         127123 non-null  object
 14  business_id   5 non-null       object
 15  name          5 non-null       object
 16  address       5 non-null       object
 17  city          5 non-null       object
 18  state         5 non-null     

In [8]:
df_business.isna().sum()

business_id          0
name                 0
address              0
city                 0
state                3
postal_code          0
latitude             0
longitude            0
stars                0
review_count         0
is_open              0
attributes       13744
categories         103
hours            23223
business_id     150341
name            150341
address         150341
city            150341
state           150341
postal_code     150341
latitude        150341
longitude       150341
stars           150341
review_count    150341
is_open         150341
attributes      150341
categories      150341
hours           150341
dtype: int64

### Valores nulos.

Al observar la existencia de columnas duplicadas cuyo contenido está compuesto en su mayor parte de valores nulos se procede a eliminarlas.

In [9]:
df_business = df_business.loc[:, ~df_business.columns.duplicated()]

Se corrobora que se hayan efectuado los cambios.

In [10]:
df_business.isna().sum()

business_id         0
name                0
address             0
city                0
state               3
postal_code         0
latitude            0
longitude           0
stars               0
review_count        0
is_open             0
attributes      13744
categories        103
hours           23223
dtype: int64

### Columna "business_id".

In [11]:
duplicados_business = df_business['business_id'].duplicated()
df_duplicados_business = df_business[duplicados_business]
df_duplicados_business.shape

(0, 14)

La columna "business_id" no presenta valores duplicados.

### Columnas "categories"

In [12]:
categories_lists = df_business['categories'].str.split(', ')

# Apilar todas las listas resultantes en una sola lista
all_categories = [category for sublist in categories_lists if sublist is not None for category in sublist]

# Obtener los valores únicos de la lista
unique_categories = set(all_categories)
print(f'La cantidad de categorías únicas es{len(unique_categories)}')

La cantidad de categorías únicas es1311


Se crea el dataframe "df_restaurants" en base a las filas que contienen el valor "restaurant" dentro de ellas con el objetivo de reducir la cantidad de filas a solo las que serán utilizadas.

In [13]:
filtro_restaurant = df_business['categories'].str.contains('restaurant', case=False, na=False)
df_restaurants = df_business[filtro_restaurant]
df_restaurants = df_restaurants.reset_index(drop=True)

In [14]:
df_restaurants.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52286 entries, 0 to 52285
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   business_id   52286 non-null  object
 1   name          52286 non-null  object
 2   address       52286 non-null  object
 3   city          52286 non-null  object
 4   state         52286 non-null  object
 5   postal_code   52286 non-null  object
 6   latitude      52286 non-null  object
 7   longitude     52286 non-null  object
 8   stars         52286 non-null  object
 9   review_count  52286 non-null  object
 10  is_open       52286 non-null  object
 11  attributes    51720 non-null  object
 12  categories    52286 non-null  object
 13  hours         45007 non-null  object
dtypes: object(14)
memory usage: 5.6+ MB


### Columna "state".

Valores nulos.

In [15]:
nulos_estado = df_business['state'].isnull().sum()
print(f'La cantidad de valores nulos presente en la columna "state" es {nulos_estado}.')

La cantidad de valores nulos presente en la columna "state" es 3.


Se realiza la imputación de valores faltantes de la columna "state" con ayuda del valor más frecuente de la columna "postal_code" para el estado correspondiente.

In [16]:
# Se genera una función lambda que será utilizada en la función de imputación.
imputacion_dict = df_restaurants.groupby('postal_code')['state'].apply(lambda x: x.mode()[0] if not x.mode().empty else None).to_dict()

# Se crea una función para llevar a cabo la imputación de datos.
def imputar_estado(row):
    if pd.isnull(row['state']):
        return imputacion_dict.get(row['postal_code'], row['state'])
    else:
        return row['state']

# Se aplica la función a cada fila del DataFrame.
df_restaurants['state'] = df_restaurants.apply(imputar_estado, axis=1)

In [17]:
nulos_estado = df_restaurants['state'].isnull().sum()
print(f'La cantidad de valores nulos presente en la columna "state" es {nulos_estado}.')

La cantidad de valores nulos presente en la columna "state" es 0.


In [18]:
valores_unicos_estado = df_restaurants['state'].unique()
estados_unicos = ', '.join(valores_unicos_estado)
conteo_estados = df_restaurants['state'].nunique()
print(f'La cantidad de estados es: {conteo_estados}. Éstos siendo {estados_unicos}')

La cantidad de estados es: 21. Éstos siendo CA, AZ, TN, MO, FL, IN, LA, PA, AB, ID, IL, NJ, NV, DE, CO, WA, HI, TX, MI, VI, VT


### Al haberse realizado una reducción de los datos a través del dataframe "df_restaurants" se usará los datos de la columna "business_id" para descartar las filas de los demás datasets que no coincidan con dicho ID.

In [25]:
business_ids = df_restaurants['business_id']

## Archivo "checkin.json"

In [19]:
checkin = []

with open(path_checkin, 'r') as archivo:
    #Se crea un loop para ir incorporando los elementos a la lista.
    for linea in archivo:
        data= json.loads(linea)
        checkin.append(data)

In [20]:
df_checkin = pd.DataFrame(checkin)
df_checkin

,business_id,date
0,---kPU91CF4Lq2-WlRu9Lw,"2020-03-13 21:10:56, 2020-06-02 22:18:06, 2020..."
1,--0iUa4sNDFiZFrAdIWhZQ,"2010-09-13 21:43:09, 2011-05-04 23:08:15, 2011..."
2,--30_8IhuyMHbSOcNWd6DQ,"2013-06-14 23:29:17, 2014-08-13 23:20:22"
3,--7PUidqRWpRSpXebiyxTg,"2011-02-15 17:12:00, 2011-07-28 02:46:10, 2012..."
4,--7jw19RH9JKXgFohspgQw,"2014-04-21 20:42:11, 2014-04-28 21:04:46, 2014..."
...,...,...
131925,zznJox6-nmXlGYNWgTDwQQ,"2013-03-23 16:22:47, 2013-04-07 02:03:12, 2013..."
131926,zznZqH9CiAznbkV6fXyHWA,2021-06-12 01:16:12
131927,zzu6_r3DxBJuXcjnOYVdTw,"2011-05-24 01:35:13, 2012-01-01 23:44:33, 2012..."
131928,zzw66H6hVjXQEt0Js3Mo4A,"2016-12-03 23:33:26, 2018-12-02 19:08:45"


In [21]:
df_checkin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131930 entries, 0 to 131929
Data columns (total 2 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   business_id  131930 non-null  object
 1   date         131930 non-null  object
dtypes: object(2)
memory usage: 2.0+ MB


### Valores nulos.

In [22]:
df_checkin.isna().sum()

business_id    0
date           0
dtype: int64

Se observa que el archivo no presenta valores nulos en ninguna de sus columnas

### Datos duplicados

In [23]:
# Se corrobora que no haya filas duplicadas en la columna "business_id".
duplicados = df_checkin['business_id'].duplicated()
duplicados_df = df_checkin[duplicados]
duplicados_df.shape

(0, 2)

### Filtro de datos.

In [26]:
# Se filtran las filas que no contengan los "business_ids" correspondientes.
df_checkin_final = df_checkin[df_checkin['business_id'].isin(business_ids)]
# Se realiza el reset del índice.
df_checkin_final = df_checkin.reset_index(drop=False, inplace=False)
df_checkin_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131930 entries, 0 to 131929
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   index        131930 non-null  int64 
 1   business_id  131930 non-null  object
 2   date         131930 non-null  object
dtypes: int64(1), object(2)
memory usage: 3.0+ MB


## Archivo "review.json"

In [4]:
import pyarrow as pa
import pyarrow.parquet as pq

Se extraen los datos del archivo "review.json" para pasarlo a formato parquet.

In [5]:
path_json = "C:\\Users\\fedez\\OneDrive\\Escritorio\\ProyectoG4-Google_Yelp\\Data\\Raw\\review.json"
path_review_parquet = "C:\\Users\\fedez\\OneDrive\\Escritorio\\ProyectoG4-Google_Yelp\\Data\\Raw\\review.parquet"

# Se lee el archivo json en fragmentos para manejar archivos grandes
chunksize = 100000  # Número de líneas por fragmento
chunks = pd.read_json(path_review, lines=True, chunksize=chunksize)

# Se crea una lista para almacenar las tablas PyArrow
tables = []

# Se procesa cada fragmento y convertirlo en una tabla PyArrow
for chunk in chunks:
    table = pa.Table.from_pandas(chunk)
    tables.append(table)

# Concatenar todas las tablas en una sola tabla
final_table = pa.concat_tables(tables)

# Guardar la tabla final en un archivo Parquet
pq.write_table(final_table, path_review_parquet)

In [7]:
# Especificamos el tamaño del trozo que queremos leer y guardar
chunksize = 1000000  # 1 millón de filas por trozo

# Abre el archivo Parquet
parquet_file = pq.ParquetFile(path_review_parquet)

# Obtén el número total de grupos de filas en el archivo Parquet
total_row_groups = parquet_file.num_row_groups

# Itera sobre el archivo Parquet y guarda cada trozo como un archivo Parquet
for i in range(0, total_row_groups):
    # Lee un trozo del archivo Parquet
    lote = parquet_file.read_row_group(i, use_pandas_metadata=True).to_pandas()

    # Guarda el trozo como un archivo Parquet
    lote.to_parquet(f"review_{i}.parquet", index=False)

    # mensaje de procesos completados por archivo
    print(f"Lote {i + 1} guardado como review_{i}.parquet")

print("Proceso completo.")

Lote 1 guardado como review_0.parquet
Lote 2 guardado como review_1.parquet
Lote 3 guardado como review_2.parquet
Lote 4 guardado como review_3.parquet
Lote 5 guardado como review_4.parquet
Lote 6 guardado como review_5.parquet
Lote 7 guardado como review_6.parquet
Proceso completo.


### Review 0.

In [11]:
df_review_0 = pd.read_parquet('C:\\Users\\fedez\\OneDrive\\Escritorio\\ProyectoG4-Google_Yelp\\Data\\Processed\\review_0.parquet')

In [12]:
df_review_0.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5,1,0,1,I've taken a lot of spin classes over the year...,2012-01-03 15:28:18
2,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3,0,0,0,Family diner. Had the buffet. Eclectic assortm...,2014-02-05 20:30:30
3,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5,1,0,1,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03
4,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4,1,0,1,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15


In [14]:
df_review_0.isna().sum()

review_id      0
user_id        0
business_id    0
stars          0
useful         0
funny          0
cool           0
text           0
date           0
dtype: int64

## Archivo "tip.json"

In [40]:
tip = [] 
#Se abre el archivo "tip.json".
with open(path_tip, encoding="utf-8") as archivo:
    # EL bucle irá incorporando los datos a la lista.
    for linea in archivo.readlines():
        tip.append(ast.literal_eval(linea))

#Se crea el dataframe "df_tip"
df_tip = pd.DataFrame(tip)

In [41]:
df_tip.head()

,user_id,business_id,text,date,compliment_count
0,AGNUgVwnZUey3gcPCJ76iw,3uLgwr0qeCNMjKenHJwPGQ,Avengers time with the ladies.,2012-05-18 02:17:21,0
1,NBN4MgHP9D3cw--SnauTkA,QoezRbYQncpRqyrLH6Iqjg,They have lots of good deserts and tasty cuban...,2013-02-05 18:35:10,0
2,-copOvldyKh1qr-vzkDEvw,MYoRNLb5chwjQe3c_k37Gg,It's open even when you think it isn't,2013-08-18 00:56:08,0
3,FjMQVZjSqY8syIO-53KFKw,hV-bABTK-glh5wj31ps_Jw,Very decent fried chicken,2017-06-27 23:05:38,0
4,ld0AperBXk1h6UbqmM80zw,_uN0OudeJ3Zl_tf6nxg5ww,Appetizers.. platter special for lunch,2012-10-06 19:43:09,0


In [42]:
df_tip.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 908915 entries, 0 to 908914
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   user_id           908915 non-null  object
 1   business_id       908915 non-null  object
 2   text              908915 non-null  object
 3   date              908915 non-null  object
 4   compliment_count  908915 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 34.7+ MB


### Valores nulos.

In [43]:
df_tip.isna().sum()

user_id             0
business_id         0
text                0
date                0
compliment_count    0
dtype: int64

Se observa que el archivo no presenta valores nulos.

### Valores duplicados.

Se corrobora que si hay filas duplicadas en base al contenido de las columnas "user_id", "text" y "date"

In [44]:
valores_duplicados = df_tip.duplicated(subset=['user_id', 'text', 'date'], keep=False)
filas_valores_duplicados = df_tip[valores_duplicados]
filas_valores_duplicados.shape

(126, 5)

Se eliminan las filas que poseen valores duplicados en las columnas "user_id", "text" y "date".

In [45]:
df_tip = df_tip.drop_duplicates(subset=['user_id', 'text', 'date'], keep='first')

In [46]:
# Se verifica que se hayan realizado los cambios.
valores_duplicados = df_tip.duplicated(subset=['user_id', 'text', 'date'], keep=False)
filas_valores_duplicados = df_tip[valores_duplicados]
filas_valores_duplicados.shape

(0, 5)

### Columna "date".

In [47]:
fecha_maxima = df_tip['date'].max()
fecha_minima = df_tip['date'].min()

print(f'La fecha mínima es {fecha_minima} y la fecha máxima es {fecha_maxima}')


La fecha mínima es 2009-04-16 13:11:49 y la fecha máxima es 2022-01-19 20:38:55


### Filtro de datos.

In [48]:
# Se filtran las filas que no contengan los "business_ids" correspondientes.
df_tip_final = df_tip[df_tip['business_id'].isin(business_ids)]
# Se realiza el reset del índice.
df_tip_final = df_tip.reset_index(drop=False, inplace=False)
df_tip_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 908848 entries, 0 to 908847
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   index             908848 non-null  int64 
 1   user_id           908848 non-null  object
 2   business_id       908848 non-null  object
 3   text              908848 non-null  object
 4   date              908848 non-null  object
 5   compliment_count  908848 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 41.6+ MB


## Archivo "user.parquet"

In [28]:
df_user = pd.read_parquet(path_user, engine='fastparquet')

In [35]:
df_user.head()

,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,average_stars,compliment_hot,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,qVc8ODYU5SZjKXVBgXdI7w,Walker,585,2007-01-25 16:47:26,7217,1259,5994,2007,"NSCy54eWehBJyZdG2iE84w, pe42u7DcCH2QmI81NX-8qA...",267,3.91,250,65,55,56,18,232,844,467,467,239,180
1,j14WgRoU_-2ZE1aw1dXrJg,Daniel,4333,2009-01-25 04:35:42,43091,13066,27281,"2009,2010,2011,2012,2013,2014,2015,2016,2017,2...","ueRPE0CX75ePGMqOFVj6IQ, 52oH4DrRvzzl8wh5UXyU0A...",3138,3.74,1145,264,184,157,251,1847,7054,3131,3131,1521,1946
2,2WnXYQFK0hXEoTxPtV2zvg,Steph,665,2008-07-25 10:41:00,2086,1010,1003,"2009,2010,2011,2012,2013","LuO3Bn4f3rlhyHIaNfTlnA, j9B4XdHUhDfTKVecyWQgyA...",52,3.32,89,13,10,17,3,66,96,119,119,35,18
3,SZDeASXq7o05mMNLshsdIA,Gwen,224,2005-11-29 04:38:33,512,330,299,"2009,2010,2011","enx1vVPnfdNUdPho6PH_wg, 4wOcvMLtU6a9Lslggq74Vg...",28,4.27,24,4,1,6,2,12,16,26,26,10,9
4,hA5lMy-EnncsH4JoR-hFGQ,Karen,79,2007-01-05 19:40:59,29,15,7,,"PBK4q9KEEBHhFvSXCUirIw, 3FWPpM7KU1gXeOM_ZbYMbA...",1,3.54,1,1,0,0,0,1,1,0,0,0,0


In [36]:
df_user.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1987897 entries, 0 to 1987896
Data columns (total 22 columns):
 #   Column              Dtype  
---  ------              -----  
 0   user_id             object 
 1   name                object 
 2   review_count        int64  
 3   yelping_since       object 
 4   useful              int64  
 5   funny               int64  
 6   cool                int64  
 7   elite               object 
 8   friends             object 
 9   fans                int64  
 10  average_stars       float64
 11  compliment_hot      int64  
 12  compliment_more     int64  
 13  compliment_profile  int64  
 14  compliment_cute     int64  
 15  compliment_list     int64  
 16  compliment_note     int64  
 17  compliment_plain    int64  
 18  compliment_cool     int64  
 19  compliment_funny    int64  
 20  compliment_writer   int64  
 21  compliment_photos   int64  
dtypes: float64(1), int64(16), object(5)
memory usage: 348.8+ MB


### Valores nulos.

In [31]:
df_user.isna().sum()

user_id               0
name                  0
review_count          0
yelping_since         0
useful                0
funny                 0
cool                  0
elite                 0
friends               0
fans                  0
average_stars         0
compliment_hot        0
compliment_more       0
compliment_profile    0
compliment_cute       0
compliment_list       0
compliment_note       0
compliment_plain      0
compliment_cool       0
compliment_funny      0
compliment_writer     0
compliment_photos     0
dtype: int64

### Valores duplicados.

In [32]:
duplicados = df_user['user_id'].duplicated()
duplicados_user = df_user[duplicados]
duplicados_user.shape

(117700, 22)

Se observa que en el dataset hay 117700 filas duplicadas las cuales serán eliminadas.

In [33]:
df_user = df_user.drop_duplicates(keep='first', inplace=False)

In [34]:
duplicados = df_user['user_id'].duplicated()
duplicados_user = df_user[duplicados]
duplicados_user.shape

(0, 22)